In [ ]:
{'data': [
    {'nickname': '关伯兰', 'remark': '关伯兰', 'user_id': 474527445}, 
    {'nickname': '🍁🍁🍁', 'remark': '🍁🍁🍁', 'user_id': 577003680}, 
], 'message': '', 'retcode': 0, 'status': 'ok'}

1.用户消息至控制器DispatcherServlet
2.DispatcherServlet进行加工处理后调用处理器映射器HandlerMapping。
3.HandlerMapping根据消息找到具体的处理器，生成响应消息返回给DispatcherServlet。
4.DispatcherServlet根据处理器Handler获取处理器适配器HandlerAdapter执行HandlerAdapter处理一系列的操作，如：参数封装，数据格式转换，数据验证等操作
5.执行处理器Handler(Controller，也叫页面控制器)。
6.Handler执行完成返回ModelAndView
7.HandlerAdapter将Handler执行结果ModelAndView返回到DispatcherServlet
8.DispatcherServlet将ModelAndView传给ViewReslover视图解析器
9.ViewReslover解析后返回具体View
10.DispatcherServlet对View进行渲染视图（即将模型数据model填充至视图中）。
11.DispatcherServlet响应用户。


In [ ]:
{
    'post_type': 'message', 
    'message_type': 'private', 
    'time': 1694183059, 
    'self_id': 2762018040, 
    'sub_type': 'friend', 
    'target_id': 2762018040, 
    'message': '你好', 
    'raw_message': '你好', 
    'font': 0, 
    'sender': {'age': 0, 'nickname': 'Aki-Polaris', 'sex': 'unknown', 'user_id': 815049548}, 
    'message_id': -2001115448, 
    'user_id': 815049548
}
message_info = {
    'post_type': 'message', 
    'message_type': 'group', 
    'time': 1694395091, 
    'self_id': 2762018040, 
    'sub_type': 'normal', 
    'anonymous': None, 
    'message': '[CQ:image,file=138fd15bdabbcfb4c3daa148555fe447.image,subType=1,url=https://gchat.qpic.cn/gchatpic_new/1352402688/830954892-2582910460-138FD15BDABBCFB4C3DAA148555FE447/0?term=255&amp;is_origin=0]', 
    'message_seq': 81567, 
    'raw_message': '[CQ:image,file=138fd15bdabbcfb4c3daa148555fe447.image,subType=1,url=https://gchat.qpic.cn/gchatpic_new/1352402688/830954892-2582910460-138FD15BDABBCFB4C3DAA148555FE447/0?term=255&amp;is_origin=0]', 
    'font': 0, 
    'group_id': 830954892, 
    'sender': {'age': 0, 'area': '', 'card': '', 'level': '', 'nickname': '离岱和葛饰应为', 'role': 'member', 'sex': 'unknown', 'title': '', 'user_id': 1352402688}, 
    'user_id': 1352402688, 
    'message_id': -185279243
}

In [ ]:
from ltp import LTP
ltp = LTP('LTP/small')
message = '明天天怎么样呀'

In [ ]:
from service.MatchSys.object_definition import Statement
from service.MatchSys.utils import import_module


class MessageAdapter(object):
    """
    This is an abstract class that represents the interface
    that all message adapters should implement.
    """
    def __init__(self, **kwargs) -> None:
        from service.MatchSys.object_definition import Statement
        from ltp import LTP

        # 初始化预处理程序
        preprocessors = kwargs.get('preprocessors', ['jionlp.clean_text'])
        self.preprocessors = []
        for preprocessor in preprocessors:
            self.preprocessors.append(import_module(preprocessor))
        
        model_path = kwargs.get('model_path', 'LTP/small')
        self.ltp = LTP(model_path)
    class AdapterMethodNotImplementedError(NotImplementedError):
        """
        An exception to be raised when an adapter method has not been implemented.
        Typically this indicates that the developer is expected to implement the
        method in a subclass.
        """

        def __init__(self, message='This method must be overridden in a subclass method.'):
            """
            Set the message for the exception.
            """
            super().__init__(message)

    def check(self, message):
        if message is None or message == '':
            raise self.ChatBotException(
                'Either a statement object or a "text" keyword '
                'argument is required. Neither was provided.'
            )
        
        return True

    def process(self, message):
        # read the message

        # Get Text message
        input_statement = self.text_process(text=message)

        # Add Other Info

        # 获取Statement
        raise self.AdapterMethodNotImplementedError()

    def text_process(self, text,**kwargs):
        """Return Search Text

        Args:
            text (_type_): _description_

        Returns:
            _type_: Statement
        """
        # 清理文本
        for preprocessor in self.preprocessors:
            text = preprocessor(text)
        kwargs['text'] = text
        # 分词
        result = self.ltp.pipeline('你觉得A怎么样', tasks = ["cws","srl"])
        kwargs['search_text'] = result.cws
        t = result.srl[0]
        for item in result.srl:
            if len(t['arguments']) > len(item['arguments']):
                t = item
        kwargs['intent'] = t

        input_statement = Statement(**kwargs)
        return input_statement

    @property
    def class_name(self):
        """
        Return the name of the current logic adapter class.
        This is typically used for logging and debugging.
        """
        return str(self.__class__.__name__)

In [ ]:
class TextMessageAdapter(MessageAdapter):
    """
    This is an abstract class that represents the interface
    that all message adapters should implement.
    """
    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)

    def process(self, message):
        # read the message

        # Get Text message
        input_statement = self.text_process(text=message)

        # Add Other Info

        # 获取Statement
        return input_statement

In [ ]:
message_adapter = TextMessageAdapter(model_path=r'D:\Code\MyLongTimeProject\A\model')

In [ ]:
from service.MatchSys.trainers import Trainer
from service.MatchSys.utils import print_progress_bar

class QATrainer(Trainer):
    """
    Allows a chat bot to be trained using a list of strings
    where the list represents a conversation.
    """

    def train(self, conversation, **kwargs):
        """
        {Q:[A1,A2...]}
        Train the chat bot based on the provided list of
        statements that represents a single conversation.
        """
        source = kwargs.get('source', 'TRAIN_DATA')
        conversation_text = kwargs.get('conversation', 'TRAIN_DATA')
        statements_to_create = []
        for index,Q in enumerate(conversation):
            if self.show_training_progress:
                print_progress_bar(
                    'QA Trainer',
                    index + 1, len(conversation)
                )
    
            statement = self.chatbot.message_adapter.process(Q)
            statement.next_id=-statement.id
            statement.conversation=conversation_text
            statement.type_of='Q'
            statement.source=source
            statement.persona='user'
            statements_to_create.append(statement)
            for A in conversation[Q]:
                statement = self.chatbot.message_adapter.process(A)
                statement.previous_id=-statement.id
                statement.conversation=conversation_text
                statement.type_of='A'
                statement.source=source
                statement.persona='bot:'+self.chatbot.name
                statements_to_create.append(statement)
        self.chatbot.storage.create_many(statements_to_create)
        self.chatbot.docvector_tool.train(statements_to_create)

In [ ]:
from service.MatchSys.match_sys import MatchSys


matchsys = MatchSys(name='Alice',model_path=r'D:\Code\MyLongTimeProject\A\model')

In [ ]:
qatrain = QATrainer(matchsys)

In [ ]:
import pandas as pd
a = pd.read_csv(r'data1.txt',
                sep="$",
                engine='python'
                )
a.head(5)
a.to_csv(r'data1.csv',)

In [ ]:
a = {'a':[1,2,3], 'b':[4,5,6], 'c':[7,8]}

In [ ]:
for index,k in enumerate(a.items()):
    print(index)
    print(k)

In [ ]:
import pandas as pd
map = {}
# data = pd.read_excel(r'C:\Users\Administrator\Documents\GitHub\QQ-Bot-And-Tool\data\FixedReply\傲娇系二次元bot词库5千词V1.2.xlsx',header=None, sheet_name=0)
data = pd.read_excel(r"data\FixedReply\Test-1K.xlsx",header=None, sheet_name=0)
for index,row in data.iterrows():
    row[0] = str(row[0]).strip()
    row[1] = str(row[1]).strip()
    if row[0] in map:
        map[row[0]].append(row[1])
    else:
        map[row[0]] = [row[1]]

In [ ]:
import pandas as pd
data = pd.read_excel(r"data\FixedReply\Test-1K.xlsx",header=None, sheet_name=0)

In [ ]:
data[0].to_list()

In [ ]:
import faiss
import numpy as np
# set HNSW index parameters
M = 64  # number of connections each vertex will have
ef_search = 32  # depth of layers explored during search
ef_construction = 64  # depth of layers explored during index construction

# initialize index (d == 128)
index = faiss.IndexHNSWFlat(768, M)
# set efConstruction and efSearch parameters
index.hnsw.efConstruction = ef_construction
index.hnsw.efSearch = ef_search
# add data to index
sentence_embeddings = np.load("data/vec_data.npy")
sentence_embeddings = sentence_embeddings.astype(np.float32)
index.add(sentence_embeddings)


In [ ]:
len(sentence_embeddings[0])

In [35]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import os
import time

class FaissSearch(object):
    def __init__(self,**kwargs):
        
        self.vector_model = SentenceTransformer('data/text2vec-base-chinese')
        self.nlist = 50
        self.hnsw_index = None
        self.signal_file_size = 10000
        self.data_d = 128
    def get_ids(self,data_list):
        ids = []
        for index,value in enumerate(data_list):
            ids.append(index)
        # 将向量处理结果存储
        ids_file = "data/Faiss/ids_data.npy"
        np.save(ids_file, np.array(ids))
        file_size = os.path.getsize(ids_file)
        print("%7.3f MB" % (file_size/1024/1024))
    def data_process_10w(self,**kwargs):
        data_save_path = kwargs.get('data_save_path',"data/Faiss/vector_data") 
        sentences = kwargs.get('data_list',None)
        # 加载模型，将数据进行向量化处理
        sentence_embeddings = self.vector_model.encode(sentences)
        # 将向量处理结果存储
        n, d = sentence_embeddings.shape
        sentence_embeddings_file_name = str(n)+"-"+str(d)+"-"+str(time.time())+"-vec_data.npy"
        np.save(os.path.join(data_save_path,sentence_embeddings_file_name), sentence_embeddings)
        file_size = os.path.getsize(os.path.join(data_save_path,sentence_embeddings_file_name))
        print(sentence_embeddings_file_name + " %7.3f MB" % (file_size/1024/1024))
    def data_process_2_vector(self,**kwargs):
        data_save_path = kwargs.get('data_save_path',"data/Faiss/vector_data") 
        if not os.path.exists(data_save_path):
            os.mkdir(data_save_path)
        data_list = kwargs.get('data_list',None)
        file_list = kwargs.get('file_list',None)
        if data_list:
            # 加载模型，将数据进行向量化处理
            sentences = data_list
            if len(sentences) <= 100000:
                self.data_process_10w(**kwargs)
            else:
                for i in range(0, len(sentences), 100000):
                    kwargs['data_list'] = sentences[i: i + 100000]
                    self.data_process_10w(**kwargs)

        elif file_list:
            """

            """
    def build_hnsw(self,vec_path='data/Faiss/vector_data'):
        files = []
        for file in os.listdir(vec_path):
            if file.endswith('.npy'):
                files.append(os.path.join(vec_path,file))

        n,d = np.load(files[0]).shape
        # # set HNSW index parameters
        # M = 64  # number of connections each vertex will have 越大搜索时间越长
        # ef_search = 32  # depth of layers explored during search 越大搜索时间越长
        # ef_construction = 64  # depth of layers explored during index construction 越大构建时间越长

        # initialize index (d == 128)
        description = 'IDMap,HNSW32,Flat'
        self.index = faiss.index_factory(d, description)
        # self.hnsw_index = faiss.IndexHNSWFlat(d, M)
        # # set efConstruction and efSearch parameters
        # self.hnsw_index.hnsw.efConstruction = ef_construction
        # self.hnsw_index.hnsw.efSearch = ef_search
        # # self.hnsw_index = faiss.IndexIDMap2(self.hnsw_index)
        
        self.add_data(files)
    def add_data(self,vec_path_list=None):
        index_embeddings = 0
        for file in vec_path_list:
            sentence_embeddings = np.load(file)
            sentence_embeddings = sentence_embeddings.astype(np.float32)
            
            self.index.add_with_ids(sentence_embeddings,ids)
            print(file + "载入完毕，数据量", len(sentence_embeddings))
            
    def search(self,sentences,topk=1):
        if isinstance(sentences,list):
            pass
        else:
            sentences = [sentences]
        sentence_embeddings = self.vector_model.encode(sentences)
        return self.hnsw_index.search(sentence_embeddings,topk), self.index.search(sentence_embeddings,topk)
faissSearch = FaissSearch()

In [31]:
import pandas as pd
data = pd.read_excel(r"data\FixedReply\傲娇系二次元bot词库5千词V1.2.xlsx",header=None, sheet_name=0)
faissSearch.data_process_2_vector(data_list=data[0].to_list())

5856-768-1700813950.7410042-vec_data.npy  17.156 MB


In [12]:
faissSearch.add_data()

载入向量数据完毕，数据量 5856


In [36]:
faissSearch.build_hnsw()

5856-768-1700813950.7410042-vec_data.npy载入完毕，数据量 5856


In [4]:
faiss.write_index(faissSearch.hnsw_index,'data/Faiss/hnws.indx')

In [37]:
hnsw_index, index = faissSearch.search(['我喜欢你'])

In [40]:
print(hnsw_index)
print(index)

(array([[2.5935717e-10]], dtype=float32), array([[52]], dtype=int64))
(array([[2.5935717e-10]], dtype=float32), array([[53]], dtype=int64))


In [ ]:
# 加载预处理数据
import numpy as np
sentences = list(map.keys())
sentence_embeddings = np.load("data/vec_data.npy")
print("载入向量数据完毕，数据量", len(sentence_embeddings))

# 使用构建向量时的模型来构建向量索引
import faiss
dimension = sentence_embeddings.shape[1]
quantizer = faiss.IndexFlatL2(dimension)
nlist = 50
index = faiss.IndexIVFFlat(quantizer, dimension, nlist)
index.train(sentence_embeddings)
index.add(sentence_embeddings)
print("建立向量索引完毕，数据量", index.ntotal)

# 尝试进行查询
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('data/text2vec-base-chinese')
print("载入模型完毕")

In [ ]:
topK = 10
search = model.encode(["好累啊"])
D, I = index.search(search, topK)


In [7]:
from PIL import Image
import numpy as np
from sklearn.decomposition import PCA

In [18]:

img = Image.open(r"D:\Data\Image\Normal\Persion\icoser.net (02).jpg")

gray_img = img.convert('L')
img_array = np.array(gray_img)
img_array = img_array / 255
pca = PCA(n_components=100)
transformed_vector = pca.fit_transform(img_array)
pca2 = PCA(n_components=100)
transformed_vector = pca2.fit_transform(transformed_vector)
print(transformed_vector.shape)
print(len(transformed_vector))

(3511, 100)
3511


In [2]:
import h5py
import argparse
import numpy as np
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input as preprocess_input_vgg
from keras.utils import image_utils
from numpy import linalg as LA
import os
import sys
from os.path import dirname


class VGGNet(object):
    def __init__(self):
        self.input_shape = (224, 224, 3)
        self.weight = 'imagenet'
        self.pooling = 'max'
        self.model_vgg = VGG16(weights=self.weight,
                               input_shape=(self.input_shape[0], self.input_shape[1], self.input_shape[2]),
                               pooling=self.pooling,
                               include_top=False)
        self.model_vgg.predict(np.zeros((1, 224, 224, 3)))

    def vgg_extract_feat(self, img_path):
        img = image_utils.load_img(img_path, target_size=(self.input_shape[0], self.input_shape[1]))
        img = image_utils.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input_vgg(img)
        feat = self.model_vgg.predict(img)
        norm_feat = feat[0] / LA.norm(feat[0])
        norm_feat = [i.item() for i in norm_feat]
        return norm_feat
    
def get_imlist(path):
    return [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.jpg')]


img_lib_path = 'D:\Data\Image\emoji'
img_index_path = './data/faiss/img_index.h5'
img_list = get_imlist(img_lib_path)
feats = []
names = []
model = VGGNet()
for i, img_path in enumerate(img_list):
    norm_feat = model.vgg_extract_feat(img_path)
    img_name = os.path.split(img_path)[1]
    feats.append(norm_feat)
    names.append(img_name)
    print("extracting feature from image No. %d %s, %d images in total" %((i+1), img_name ,len(img_list)))
feats = np.array(feats)
h5f = h5py.File(img_index_path, 'w')
h5f.create_dataset('dataset_1', data = feats)
h5f.create_dataset('dataset_2', data = np.string_(names))
h5f.close()

1/1 [==============================] - 0s 143ms/step
extracting feature from image No. 1 0486A7467002C3EA08331A7C02CCBA04.jpg, 12 images in total
1/1 [==============================] - 0s 147ms/step
extracting feature from image No. 2 1651812572141.jpg, 12 images in total
1/1 [==============================] - 0s 185ms/step
extracting feature from image No. 3 1651812581758.jpg, 12 images in total
1/1 [==============================] - 0s 163ms/step
extracting feature from image No. 4 32F31CDB9BA8C8F831F7D3BC1EC858E6.jpg, 12 images in total
1/1 [==============================] - 0s 144ms/step
extracting feature from image No. 5 75BE6EECCC48ACA7C2F07EE34B5A4888.jpg, 12 images in total
1/1 [==============================] - 0s 152ms/step
extracting feature from image No. 6 8DECDED8768C8D147C800E8DECDED8768C8D147C800E0C012313B88DECDED8768C8D147C800E0C012313B88DECDED8768C8D147C800E0C012313B88DECDED8768C8D147C800E0C012313B88DECDED8768C8D147C800E0C012313B88DECDED8768C8D147C800E0C012313B0C0123

In [ ]:
# -*- coding: utf-8 -*-

import argparse
from service.vggnet import VGGNet
from service.numpy_retrieval import NumpyRetrieval
from service.faiss_retrieval import FaissRetrieval
from service.es_retrieval import ESRetrieval
from service.milvus_retrieval import MilvusRetrieval
import os
import sys
from os.path import dirname
BASE_DIR = dirname(os.path.abspath(__file__))
sys.path.append(BASE_DIR)


class RetrievalEngine(object):

    def __init__(self, index_file, db_name):
        self.index_file = index_file
        self.db_name = db_name
        self.numpy_r = self.faiss_r = self.es_r = self.milvus_r = None

    def get_method(self, m_name):
        m_name = "%s_handler" % str(m_name)
        method = getattr(self, m_name, self.default_handler)
        return method

    def numpy_handler(self, query_vector, req_id=None):
        # numpy计算
        if self.numpy_r is None:
            self.numpy_r = NumpyRetrieval(self.index_file)
        return self.numpy_r.retrieve(query_vector)

    def faiss_handler(self, query_vector, req_id=None):
        # faiss计算
        if self.faiss_r is None:
            self.faiss_r = FaissRetrieval(self.index_file)
        return self.faiss_r.retrieve(query_vector)

    def es_handler(self, query_vector, req_id=None):
        # es计算
        if self.es_r is None:
            self.es_r = ESRetrieval(self.db_name, self.index_file)
        return self.es_r.retrieve(query_vector)

    def milvus_handler(self, query_vector, req_id=None):
        # milvus计算
        if self.milvus_r is None:
            self.milvus_r = MilvusRetrieval(self.db_name, self.index_file)
        return self.milvus_r.retrieve(query_vector)

    def default_handler(self, query_vector, req_id=None):
        return []


query_img = "D:\Data\Image\emoji\-29af59a4b1fcbc27.gif"


    parser.add_argument("--index_file", type=str, default=os.path.join(BASE_DIR, 'index', 'train.h5'), help="index file path.")
    parser.add_argument("--db_name", type=str, default='image_retrieval', help="database name.")
    parser.add_argument("--engine", type=str, default='numpy', help="retrieval engine.")
    args = vars(parser.parse_args())
# 1.图片推理
model = VGGNet()
query_vector = model.vgg_extract_feat(query_img)
# 2.图片检索
re = RetrievalEngine(img_index_path, args["db_name"])
result = re.get_method(args["engine"])(query_vector, None)
print(result)

